# Extracting early neurologic deterioriation
Def: Rise in NIHSS of min 2 or 4 points in the first 72h

In [ ]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
imputed_missing_df_path = '/Users/jk1/temp/treatment_effects/imputed_missing_df.pkl'

In [ ]:
with open(imputed_missing_df_path, 'rb') as f:
    imputed_missing_df = pickle.load(f)

In [ ]:
imputed_missing_df.case_admission_id.unique()

In [ ]:
# plot value for NIHSS sample_label over realive_sample_date_hourly_cat
for cid in imputed_missing_df.case_admission_id.unique():
    temp = imputed_missing_df[(imputed_missing_df.sample_label == 'median_NIHSS') & (imputed_missing_df.case_admission_id == cid)]
    ax = sns.scatterplot(x='relative_sample_date_hourly_cat', y='value', data=temp)
    ax.set_title(cid)
    ax.set_ylim(0, 42)
    plt.show()

In [ ]:
end_df = imputed_missing_df[(imputed_missing_df.sample_label == 'median_NIHSS')]

In [ ]:
# verify that relative_sample_date_hourly_cat is sorted
assert end_df.groupby('case_admission_id')['relative_sample_date_hourly_cat'].is_monotonic_increasing.all()

In [ ]:
# for every case_admission_id / timebin combination, substract previous value from current value
end_df['delta'] = end_df.groupby(['case_admission_id'])['value'].diff()
end_df['delta'] = end_df['delta'].fillna(0)

In [ ]:
# minimum up to current timebin
end_df['best_prior_state'] = end_df.groupby(['case_admission_id'])['value'].cummin()
end_df['delta_to_best_prior_state'] = end_df['value'] - end_df['best_prior_state']

In [ ]:
end_df['start_state'] = end_df.groupby(['case_admission_id'])['value'].transform('first')
end_df['delta_to_start_state'] = end_df['value'] - end_df['start_state']

In [ ]:
end_df['delta_at_next_ts'] = end_df.groupby(['case_admission_id'])['delta'].shift(-1)
end_df['delta_to_best_prior_state_at_next_ts'] = end_df.groupby(['case_admission_id'])['delta_to_best_prior_state'].shift(-1)
end_df['delta_to_start_state_at_next_ts'] = end_df.groupby(['case_admission_id'])['delta_to_start_state'].shift(-1)

In [ ]:
# plot value, delta, delta_to_best_prior_state and delta_to_start_state over relative_sample_date_hourly_cat

temp = end_df[end_df.case_admission_id == '1005798_9217']

fig, ax = plt.subplots(4, 1, figsize=(10, 10))
sns.scatterplot(x='relative_sample_date_hourly_cat', y='value', data=temp, ax=ax[0], color='blue')

sns.scatterplot(x='relative_sample_date_hourly_cat', y='delta', data=temp, ax=ax[1], color='red', alpha=0.5)
sns.lineplot(x='relative_sample_date_hourly_cat', y='delta_at_next_ts', data=temp, ax=ax[1], color='pink', alpha=0.5)
sns.scatterplot(x='relative_sample_date_hourly_cat', y='delta_to_best_prior_state', data=temp, ax=ax[2], color='green')
sns.scatterplot(x='relative_sample_date_hourly_cat', y='delta_to_start_state', data=temp, ax=ax[3], color='orange')


In [ ]:
end_df['end_at_next_ts'] = (end_df['delta_to_start_state_at_next_ts'] >= 2).astype(int)
end_df['end'] = (end_df['delta_to_start_state'] >= 2).astype(int)

In [ ]:
end_df